In [13]:
"""
PICA Vergleichstool (Tkinter GUI)
==================================

Beschreibung:
-------------
Dieses Tool vergleicht bibliografische Daten aus einer Excel-Datei mit Datensätzen im PICA-Format
(z. B. aus einem Datendump, erzeugt mit dem Tool `pica`).

Das Ziel ist es, Felder wie Titel, Signatur, Standort usw. aus der Excel-Tabelle mit den entsprechenden
Feldern im PICA-Datensatz für eine bestimmte IDN + EPN zu vergleichen.

Funktionalität:
---------------
1. Excel-Datei laden:
   - Beliebige Excel-Datei mit bibliografischen Daten (z. B. Titel, Jahr, IDN, EPN etc.)
   - Spalten werden über Dropdown-Menüs PICA-Feldern zugeordnet

2. PICA-Datendump wählen:
   - Auswahl einer .dat-Datei mit PICA-Datensätzen (UTF-8, eine Zeile pro Feld)
   - Die Datensätze werden über `pica filter` zeilenweise extrahiert

3. Vergleich starten:
   - Für jede Zeile in der Excel-Tabelle wird:
     - Die IDN gesucht
     - Die passende EPN innerhalb der IDN gefunden (Exemplarblock)
     - Alle zugeordneten Felder mit den PICA-Werten verglichen
     - Ergebnis als ✅ (gleich) oder ❌ (abweichend) im Textfeld angezeigt

4. Ergebnis:
   - Übersichtliche Vergleichsausgabe für jede IDN/EPN-Kombination
   - Ideal zur Datenkontrolle, Korrekturprüfung oder Vorbereitung für Migrationsprojekte

Anwendung:
----------
1. Skript starten:         `python dein_skript.py`
2. "Excel-Datei laden":    Excel-Datei auswählen und Spalten zuordnen
3. "PICA-Datendump wählen": Dump-Datei auswählen (.dat, UTF-8)
4. "Vergleichen":          Vergleich starten und Ergebnisse prüfen

Beispiele für Feldzuordnung:
-----------------------------
  IDN            → 003@.0
  EPN            → 203@.0
  Signatur       → 209A.a
  Standort       → 209A.f
  Standortnotation → 209A.g
  AKZ            → 209C.a
  Titel          → 021A.a
  Jahr_a         → 011@.a
  Satzart        → 0500.a

Voraussetzungen:
----------------
- Python 3.x
- Abhängigkeiten: tkinter, pandas, PyYAML, unicodedata
- Das CLI-Tool `pica` muss installiert und im PATH verfügbar sein
- UTF-8-kodierter Datendump aus Aleph oder einem ähnlichen System

Hinweise:
---------
- Feldvergleiche sind normalisiert (z. B. Unicode-Normalisierung, Trimmen)
- Bei mehrfachen Vorkommen von Feldern (z. B. 209A in /XX-Blöcken) wird nach EPN selektiert
- Falsche oder fehlende Zuordnung führt zu Fehlermeldungen
"""




import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import subprocess
import unicodedata
import yaml
import os
import re
from collections import defaultdict

class PicaComparerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("PICA Vergleich")

        self.mapping = self.load_mapping("mapping.yaml")
        self.label_to_field = {k: v for k, v in self.mapping.items() if v}
        self.field_to_label = {v: k for k, v in self.label_to_field.items()}

        self.df = None
        self.column_mappings = {}
        self.pica_dump_path = None  # Hier wird der Pfad zum Datendump gespeichert

        self.setup_ui()

    def load_mapping(self, filepath):
        with open(filepath, "r", encoding="utf-8") as f:
            return yaml.safe_load(f)

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(padx=10, pady=10)

        # Button Excel laden
        tk.Button(frame, text="Excel-Datei laden", command=self.load_excel).grid(row=0, column=0, sticky="w")

        # Button für PICA-Datendump auswählen
        tk.Button(frame, text="PICA-Datendump wählen", command=self.select_pica_dump).grid(row=0, column=1, sticky="w", padx=10)

        # Frame für Spalten-Mapping
        self.mapping_frame = tk.Frame(frame)
        self.mapping_frame.grid(row=1, column=0, columnspan=2, pady=10, sticky="w")

        # Button Vergleich starten
        tk.Button(frame, text="Vergleichen", command=self.compare).grid(row=2, column=0, columnspan=2)

        # Textfeld Ergebnis
        self.result_text = tk.Text(frame, height=20, width=120)
        self.result_text.grid(row=3, column=0, columnspan=2, pady=10)

        # Label für ausgewählten PICA-Dump-Pfad anzeigen
        self.pica_path_label = tk.Label(frame, text="Kein PICA-Datendump ausgewählt")
        self.pica_path_label.grid(row=4, column=0, columnspan=2, sticky="w")

    def load_excel(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if not file_path:
            return

        self.df = pd.read_excel(file_path, dtype=str).fillna("")

        # Mapping Frame neu befüllen
        for widget in self.mapping_frame.winfo_children():
            widget.destroy()

        self.column_mappings.clear()

        for i, column in enumerate(self.df.columns):
            tk.Label(self.mapping_frame, text=column).grid(row=i, column=0, sticky="w")
            combo = ttk.Combobox(self.mapping_frame, values=["nichts zuordnen"] + list(self.label_to_field.keys()))
            # Versuche, automatisch passendes Mapping vorzuschlagen (wenn Spaltenname exakt einem Label entspricht)
            combo.set(column if column in self.label_to_field else "nichts zuordnen")
            combo.grid(row=i, column=1)
            self.column_mappings[column] = combo

    def select_pica_dump(self):
        file_path = filedialog.askopenfilename(title="PICA-Datendump auswählen", filetypes=[("Datendump files", "*.dat"), ("Alle Dateien", "*.*")])
        if file_path:
            self.pica_dump_path = file_path
            self.pica_path_label.config(text=f"PICA-Datendump: {file_path}")

    
    def split_into_exemplar_blocks(self, lines):
        blocks = {}
        for line in lines:
            if "/" not in line:
                continue
            try:
                field, rest = line.split(" ", 1)
                if "/" in field:
                    base, block = field.split("/")
                    blocks.setdefault(block, []).append((base, rest.strip()))
            except ValueError:
                continue
        return blocks

    def find_exemplar_block_by_epn(self, blocks, target_epn):
        for block_id, entries in blocks.items():
            for base, content in entries:
                if base == "203@":
                    subfields = content.split("")
                    for sub in subfields:
                        if sub.startswith("0") and sub[1:] == target_epn:
                            return entries
        return None

    def run_pica_query(self, idn):
        if not self.pica_dump_path:
            messagebox.showerror("Fehler", "Bitte zuerst einen PICA-Datendump auswählen.")
            return []

        cmd = f'pica filter -s "003@.0 == \\"{idn}\\"" "{self.pica_dump_path}"'
        print(f"🔍 Starte PICA-Abfrage: {cmd}")
        try:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, encoding="utf-8")
            lines = result.stdout.splitlines()

            print(f"📄 Rohdaten für IDN {idn}:")
            for line in lines:
                print(line)

            return lines
        except Exception as e:
            print(f"[Fehler bei Abfrage von IDN {idn}: {e}]")
            return []

    def normalize_str(self, s):
        if s is None:
            return ""
        s = unicodedata.normalize("NFC", str(s).strip())
        s = re.sub(r"\s+", " ", s)  # ersetzt mehrere Leerzeichen durch eines
        return s
    
    def parse_stammdaten(self, lines):
        stammdaten = {}
        for line in lines:
            if "/" not in line:
                try:
                    field, rest = line.split(" ", 1)
                    stammdaten[field] = rest.strip()
                except ValueError:
                    continue
        return stammdaten

    def parse_exemplar_blocks(self, lines):
        blocks = defaultdict(dict)
        for line in lines:
            match = re.match(r"^(\d{3}[A-Z])(?:/(\d{2}))?\s(.+)", line)
            if match:
                field, suffix, content = match.groups()
                suffix = suffix or "00"  # kein /XX → allgemeiner Block
                blocks[suffix][field] = content
        return blocks
    
    
    def extract_field_from_block(self, block_entries, pica_key):
        if '.' not in pica_key:
            return ""

        field, sub = pica_key.split('.', 1)

        # Finde alle Inhalte mit diesem Feldnamen (mehrere möglich!)
        values = []
        for f, line in block_entries:
            if f != field:
                continue
            values.extend([part[1:] for part in line.split("") if part.startswith(sub)])

        return "; ".join(values)
    
    def compare(self):
        if self.df is None:
            messagebox.showerror("Fehler", "Keine Excel-Datei geladen.")
            return

        if not self.pica_dump_path:
            messagebox.showerror("Fehler", "Bitte zuerst einen PICA-Datendump auswählen.")
            return

        self.result_text.delete(1.0, tk.END)

        idn_column = self.get_column_by_mapping("IDN")
        if not idn_column:
            messagebox.showerror("Fehler", "Keine Spalte für IDN zugeordnet.")
            return

        for _, row in self.df.iterrows():
            idn = row[idn_column]
            if not idn:
                continue

            pica_lines = self.run_pica_query(idn)

            # Stammdaten & Exemplardaten als Listen von Tupeln (field, content)
            full_record = [(line.split(" ", 1)[0], line.split(" ", 1)[1]) for line in pica_lines if " " in line]
            exemplar_blocks = self.split_into_exemplar_blocks(pica_lines)

            epn_column = self.get_column_by_mapping("EPN")
            target_epn = self.normalize_str(row[epn_column]) if epn_column else ""

            exemplar_block = self.find_exemplar_block_by_epn(exemplar_blocks, target_epn)

            self.result_text.insert(tk.END, f"\n📌 IDN {idn}:\n")

            if not exemplar_block:
                self.result_text.insert(tk.END, f"  ❌ EPN '{target_epn}' nicht im Dump gefunden\n")
                continue

            for col, combo in self.column_mappings.items():
                label = combo.get()
                if label == "nichts zuordnen" or label not in self.label_to_field:
                    continue

                pica_key = self.label_to_field[label]
                excel_val = row[col]
                excel_val_norm = self.normalize_str(excel_val)

                exemplar_fields = {"209A", "208@", "203@", "209B", "209C", "245Y", "245Z"}  # ggf. erweitern

                if pica_key.split(".")[0] in exemplar_fields:
                    pica_val = self.extract_field_from_block(exemplar_block, pica_key)
                else:
                    pica_val = self.extract_field_from_block(full_record, pica_key)

                pica_val_norm = self.normalize_str(pica_val)

                if excel_val_norm != pica_val_norm:
                    self.result_text.insert(
                        tk.END,
                        f"  ❌ {label} (Excel: '{excel_val}' ≠ PICA: '{pica_val}')\n"
                    )
                else:
                    self.result_text.insert(
                        tk.END,
                        f"  ✅ {label} (Excel: '{excel_val}' = PICA: '{pica_val}')\n"
                    )

    def get_column_by_mapping(self, label):
        for col, combo in self.column_mappings.items():
            if combo.get() == label:
                return col
        return None

if __name__ == "__main__":
    root = tk.Tk()
    app = PicaComparerApp(root)
    root.mainloop()
